## Fit model to classify sentences by smart similarity


### Load csv

In [2]:
import sys, os, re, unidecode, random, gensim, logging
import pandas as pd
from pprint import pprint
sys.path.append('/home/od13/addons/tender_cat/libcat/mytextpipe/')
from mytextpipe import corpus
from uk_stemmer import UkStemmer
from string import punctuation 
from tqdm import tqdm
from IPython.display import clear_output
import joblib 
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
import math
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.text import TfidfTransformer


# Temp
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)


class DataPipeline:
    
    def __init__(self, data_folder=None, trained_folder=None):
        self.data_folder = data_folder
        self.trained_folder = trained_folder
        self.model_path = os.path.join(self.trained_folder, 'deep_sim.mod')
        
        
        
        self.ngram = None
        self.vectorizer = None
        self.train_column = 'norm_text'
        self.train_df = None
        self.label_stat = None

        self.corp = corpus.HTMLCorpusReader('.', stemmer=UkStemmer(), clean_text=True, language='russian')
        
        # Add stop_words
        self.stop_words = self.corp.stop_words.copy()
        self.stop_words += ['в', 'на', 'за', 'із', 'та', 'що', 'як', 'грн', 'гривен', 'або', 'якщо', '2020',
                    'до', 'про', 'як', 'так', 'для', 'від', 'інш', 'щод', 'іх', 'над', 'цьог', 'дат', 'шт', 'чи', 
                    'мм', '10', 'також', 'під_час', 'при', 'коп', 'мож', 'ма', 'ціє', 'по', 'післ', 'без', 
                    'цим', 'наступн', 'це', '20', 'ус', 'сво', 'час', 'тощ', 'номер', 'місц', 'одн', 'под', 'не_пізніш', 
                    '12', 'дiаметр', 'менш', 'бут', 'окрем', 'будьяк', 'під', 'ум', 'учасник', 'дня',
                    '[', ']', 'підпис', 'печатк', 'уповноважено', 'особ', 'раз', 'зі', 'isregistry_keywords', 'рок', 'рік']
        print('Loaded {} stop words'.format(len(self.stop_words)))
        
        # Add stop_texts
        self.stop_text = []
        if 1==1:
            for root, dirs, files in os.walk(self.trained_folder):
                for f in files: 
                    if f.endswith("stop_text.txt"):
                        fname = os.path.join(self.trained_folder, f)
                        with open(fname, "r") as file:
                            for line in file:
                                line_lst = list(line.strip().split(" "))
                                txt = ' '.join(line_lst[1:])
                                self.stop_text.append(txt)
                            print('Loaded {} stop texts from {}'.format(len(self.stop_text), fname))
            self.stop_text = list(set(self.stop_text))
            print('Total {} stop texts is now in piplene'.format(len(self.stop_text)))

    
    def is_stop_text(self, txt):
        return txt in self.stop_text
    
    def is_stop_word(self, word):
        if len(word)<=1:
            return True
        if word in ['.', ',', ';', ':', '!', '?']:
            return True
        if word in self.stop_words:
            return True

    def nice_word_form(self, word):
        # Some special cases
        nice_dct = {
            'тендерно': 'тендерн',
            'тендерні': 'тендерн',
            'електронні': 'електронн',
            'договор': 'договір',
            'статт_17': 'ст17',
            'юридично': 'юридичн',
            'юридични': 'юридичн',
            'антикорупціино': 'антикорупціин',
            'банківсько': 'банківськ',
            'будь-яко': 'будь-як',
            'будь-які': 'будь-як',
            'будівельно': 'будівельн',
            'бюджетн': 'бюджет',
            'вартост': 'вартіст',
            'вважают': 'вваж',
            'вважаєт': 'вваж',
            'вимагал': 'вимаг',
            'вимагают': 'вимаг',
            'вимагаєт': 'вимаг',
            'вимог': 'вимаг',
            'випробувальн': 'випробуван',
            'випробуванн': 'випробуван',
            'виробник': 'виробн',
            'виробництв': 'виробн',
            'виробнич': 'виробн',
            'відповідальност': 'відповідальн',
            'відповідальніст': 'відповідальн',
            'державно': 'державн',
        }
        return nice_dct[word] if word in nice_dct else word
    
    def normalize(self, txt):
    
        txt = txt.strip()

        url_pattern = r'https?://\S+|http?://\S+|www\.\S+'
        txt = re.sub(pattern=url_pattern, repl=' ', string=txt)

        #number_pattern = r'\d+'
        #txt = re.sub(pattern=number_pattern, repl="nmbr", string=txt)
        #txt = re.sub(pattern=number_pattern, repl=" ", string=txt)

        single_char_pattern = r'\s+[a-zA-Z]\s+'
        txt = re.sub(pattern=single_char_pattern, repl=" ", string=txt)

        txt = gensim.utils.decode_htmlentities(gensim.utils.deaccent(txt))

        space_pattern = r'\s+'
        txt = re.sub(pattern=space_pattern, repl=" ", string=txt)

        return [self.nice_word_form(x) 
                for x in self.corp.text_to_words(txt) 
                if not self.is_stop_word(x)]
    
    def text_src(self, source_df, column='norm_text'):
        # Generator for reading texts of selected column
        # Return list of words
        for index, row in source_df.iterrows():
            txt = str(row[column]).strip()
            if txt:
                yield txt.split()
    
    def calc_word_count(self, source_df, column='norm_text'):
        word_list = []
        for txt in self.text_src(source_df, column):
            for word in txt:
                word_list.append(word)
        word_df = pd.DataFrame(word_list, columns=['word'])
        self.word_count = word_df['word'].nunique()
        return word_df

    def print_most_freq(self, source_df, column, top, reverse=True):
        _freq_word_dct = dict(source_df[column].value_counts())
        for w in sorted(_freq_word_dct, key=_freq_word_dct.get, reverse=reverse):
            print(_freq_word_dct[w], w)
            top -= 1
            if not top:
                break    
                
    def generate_ngrams(self, source_df, n, min_count=10, threshold=20):
        gram_model_filepath = os.path.join(self.trained_folder, str(n)+'gram.mod')
        column_name = 'norm_text' if n==2 else str(n)+'gram_text'
        prev_column_name = 'norm_text' if (n-1)==1 else str((n-1))+'gram_text'
        
        gram_model = Phrases(self.text_src(source_df, prev_column_name), min_count, threshold)
        gram_list = []
        for txt in self.text_src(source_df, prev_column_name):
            gram_txt = gram_model[txt]
            for word in gram_txt:
                if len(word.split('_'))==n:
                    gram_list.append(word)
        gram_df = pd.DataFrame(gram_list, columns=['word'])
        source_df[str(n)+'gram_text'] = source_df.apply(lambda row: ' '.join(gram_model[row[prev_column_name].split()]), axis=1)
        
        if not prev_column_name == 'norm_text':
            del source_df[prev_column_name]
            
        self.ngram = gram_model
        self.train_column = column_name
        return gram_df 
    
    def text_src_to_tagged_docs(self, source_df, txt_column='norm_text'):
        for index, row in source_df.iterrows():
            word_lst = str(row[txt_column]).split()
            if word_lst:
                if len(word_lst) <= 1:
                    continue
                else:
                    yield gensim.models.doc2vec.TaggedDocument(word_lst, [row['doc2vec_uid']])
    
    def ivec(self, word_list, epochs=850, alpha=0.025):
        if self.vectorizer:
            return self.vectorizer.infer_vector(word_list, epochs=epochs)
        else:
            return []
    
    def most_similar(self, txt, topn=30):
        
        similars = []
        
        if self.ngram:
            word_list = self.ngram[self.normalize(txt)]
        else:
            word_list = self.normalize(txt)
        
        if len(word_list) == 1:
            similars.append({
                'id': 0,
                'label_name': '?',
                'label_id': 1001,
                'text': txt,
                'train_text': '',
                'sim': 0,
            })
            return similars
        
        txt_vector = self.ivec(word_list)
        sims = self.vectorizer.docvecs.most_similar([txt_vector], topn=topn)

        for doc_id, sim in sims:
            found_rows = self.train_df[self.train_df.doc2vec_uid == doc_id]
            similars.append({
                'id': doc_id,
                'label_name': found_rows['label_name'],
                'label_id': found_rows['label_id'],
                'text': found_rows['text'],
                'train_text': found_rows[self.train_column],
                'sim': sim,
            })
        
        return similars
    
    def save(self):
        joblib.dump({
            'ngram': self.ngram,
            'vectorizer': self.vectorizer,
            'stop_words': self.stop_words,
            'stop_text': self.stop_text,
            'train_column': self.train_column,
            'train_df': self.train_df,
            'label_stat': self.label_stat,
        }, self.model_path)
        print('Model saved to {}'.format(self.model_path))
            
    def load(self):
        model_dct = joblib.load(self.model_path)
        self.ngram = model_dct['ngram']
        self.vectorizer = model_dct['vectorizer']
        self.stop_words = model_dct['stop_words']
        self.stop_text = model_dct['stop_text']
        self.train_column = model_dct['train_column']
        self.train_df = model_dct['train_df']
        self.label_stat = model_dct['label_stat']
        print('Model loaded from {}'.format(self.model_path))
        
    def update_label_stat(self):
        
        tdf = self.train_df
        label_freq = {}
        label_names = {}
        label_norm_freq = {}
        total_cnt = 0
        max_count = 0
        for i, row in tdf.groupby('label_id').nunique().iterrows():
            cnt = row[self.train_column]
            label_freq[i] = cnt
            total_cnt += cnt
            
            if cnt > max_count:
                max_count = cnt
            
            label_names[i] = tdf.loc[tdf['label_id'] == i, 'label_name'].iloc[0]
            # calc unique texts count instead

        for label_id in label_freq:
            label_norm_freq[label_id] = label_freq[label_id]/max_count
            
        self.label_stat = {
            'freq': label_freq,
            'norm_freq': label_norm_freq,
            'name': label_names,
            'total_cnt': total_cnt,
        }
    

    def text_label(self, txt, stack=20, sim_pow=3, debug=False):
        
        sims = self.most_similar(txt, topn=stack)
        
        labels_cnt = {}
        labels_sim = {}
        
        for sim in sims:
            try:
                label_id = sim['label_id'].iloc[0]
            except:
                continue
            
            if label_id in labels_cnt: 
                labels_cnt[label_id] += 1 
                labels_sim[label_id] += math.pow(sim['sim'], sim_pow)
            else:
                labels_cnt[label_id] = 1 
                labels_sim[label_id] = math.pow(sim['sim'], sim_pow)
        
        labels_score = {}
        
        for label_id in labels_cnt:
            # Fraction of particular label set
            norm_freq = labels_cnt[label_id]/self.label_stat['freq'][label_id]
            total_norm_freq = norm_freq #/self.label_stat['norm_freq'][label_id]
            
            # Look at similarity
            score = total_norm_freq*labels_sim[label_id]
            
            labels_score[label_id] = score
            
        winner_id = sorted(labels_score, key=labels_score.get, reverse=True)[0]
        
        vals = {
            'label_id': winner_id,
            'label_name': self.label_stat['name'][winner_id],
            
        }
        
        if debug:
            sims_name = {}
            for w in sorted(labels_score, key=labels_score.get, reverse=True):
                sims_name[self.label_stat['name'][w]] = labels_score[w]
            vals.update({
                'sims': labels_score, 
                'sims_named': sims_name, 
                'sim': labels_score[winner_id]
            })
        
        return vals
 
        
    def fit(self, data_folder=None, trained_folder=None, a=None):

        ################################################
        # Load csv
        csvs = []
        for root, dirs, files in os.walk(self.data_folder):
            for f in files: 
                if f.endswith(".csv"):
                    try: 
                        csv = pd.read_csv(os.path.join(self.data_folder, f))
                        csvs.append(csv)
                    except (FileExistsError, IOError, pd.errors.EmptyDataError) as e:
                        _logger.error('{}: {}'.format(f,e))

        df = pd.concat(csvs,ignore_index=True)
        df.fillna(value = {'label_id': 1001, 'label_name': '?'}, inplace=True)
        df.label_id = df.label_id.astype(int)

        print('Loaded {} training texts from {} files'.format(df.shape[0], len(set(df['file'].dropna()))))

        ################################################
        # Characters count
        df['chr_count'] = df.apply(lambda row: len(row['text']), axis=1)

        stat = {'chr_count_max': df['chr_count'].max(), 'chr_count_min': df['chr_count'].min() }

        print('Character counts in texts: min {}, max {},  mean {}\n'.format(stat['chr_count_min'], stat['chr_count_max'], df['chr_count'].mean()))

        ################################################
        # Delete too long sentences, max chr count -1
        start_len = df.shape[0]
        df = df.drop(df[df.chr_count >= stat['chr_count_max']-1].index)
        print('Deleted {} long texts'.format(start_len-df.shape[0]))

        if 0==1:
            top_n = 5
            print('\nTop {} most long now: '.format(top_n))
            print(df.sort_values('chr_count', ascending = False)['text'].head(top_n))

        ################################################
        # Delete too short sentences, chr count == min
        start_len = df.shape[0]
        max_chr_count = max(3, stat['chr_count_min'])
        df = df.drop(df[df.chr_count <= max_chr_count].index)
        print('Deleted {} short texts'.format(start_len-df.shape[0]))

        if 0==1:
            top_n = df.shape[0]//50
            print('\nTop {} most short now: '.format(top_n))
            print(df.sort_values('chr_count', ascending = True)['text'].head(top_n))
            
        ##############################################    
        # Generate normalized text, iter 1
        df['norm_text'] = df.apply(lambda row: str(' '.join(self.normalize(row['text'])).strip()), axis=1)
        
        # We may unload most frequent texts to disk here
        
        ##############################################    
        # Delete stop_texts
        start_len = df.shape[0]
        df = df.drop(df[df['norm_text'].isin(self.stop_text)].index)
        print('Deleted {} stop-texts from training corpus'.format(start_len-df.shape[0]))           

        
        ##############################################    
        # Words in texts
        word_list = []
        for txt in self.text_src(df,'norm_text'):
            for word in txt:
                word_list.append(word)
        word_df = pd.DataFrame(word_list, columns=['word'])
        print('Found {} unique tokens in texts'.format(word_df['word'].nunique()))

        # if 1==1:
        #     top_n = 300
        #     print('\nTop {} most frequent tokens:'.format(top_n))
        #     self.print_most_freq(word_df, 'word', top_n)

        if 0==1:
            top_n = 200
            print('\nTop {} rare tokens:\n'.format(top_n))
            self.print_most_freq(word_df, 'word', top_n, reverse=False)
            
        ##############################################    
        # Add rare words to stop list
        freq_word_dct = dict(word_df['word'].value_counts())
        cnt = 0            
        for w in freq_word_dct:
            if freq_word_dct[w] == 1:
                self.stop_words.append(w)
                cnt += 1
        print('{} rare words added to stop list'.format(cnt))

        ##############################################    
        # Remove stop words from norm texts, normalize text, iter 2
        df['norm_text'] = df.apply(lambda row: ' '.join(self.normalize(row['text'])), axis=1)
        print('Texts normalized again, stop-words removed from texts')
        word_df  = self.calc_word_count(df, 'norm_text')
        print('Now {} tokens in vocabulary'.format(self.word_count)) 
        
        ##############################################    
        # N-grams
        # 2-grams
        gram_df = self.generate_ngrams(df, 2, min_count=20, threshold=10)
        print('\n{} 2-grmas found'.format(gram_df['word'].nunique()))     
        word_df  = self.calc_word_count(df, '2gram_text')
        print('Now {} tokens in vocabulary'.format(self.word_count)) 
        if 0==1:
            top_n = 100
            print('\nTop {} most frequent 2-grams:\n'.format(top_n))
            self.print_most_freq(gram_df, 'word', top_n)

        if 1==1:
            # 3-grams
            gram_df = self.generate_ngrams(df, 3, min_count=10, threshold=30)
            print('\n{} 3-grmas found'.format(gram_df['word'].nunique()))         
            word_df = self.calc_word_count(df, '3gram_text')
            print('Now {} tokens in vocabulary'.format(self.word_count)) 
            if 0==1:
                top_n = 50
                print('\nTop {} most frequent 3-grams:\n'.format(top_n))
                self.print_most_freq(gram_df, 'word', top_n)

        if 0==1:
            # 4-grams
            gram_df = self.generate_ngrams(df, 4, min_count=7, threshold=40)
            print('\n{} 4-grmas found'.format(gram_df['word'].nunique()))         
            word_df = self.calc_word_count(df, '4gram_text')
            print('Now {} tokens in vocabulary'.format(self.word_count)) 
            if 1==1:
                top_n = 10
                print('\nTop {} most frequent 4-grams:\n'.format(top_n))
                self.print_most_freq(gram_df, 'word', top_n)

        
        if 0==1:
            # Show words in abc order
            top_n = 2500
            print('\nTop {} most frequent words:'.format(top_n))
            word_df = self.calc_word_count(df, '2gram_text')
            word_dct = dict(word_df['word'])
            words = []
            for w in sorted(word_dct, key=word_dct.get, reverse=False):
                if not word_dct[w] in words:
                    words.append(word_dct[w])
                    print(word_dct[w])
                    top_n -= 1
                    if not top_n:
                        break    
                        
        ##############################################    
        # Doc2vec
        df['doc2vec_uid'] = range(1, len(df.index)+1)
        train_corpus = list(self.text_src_to_tagged_docs(df, txt_column=self.train_column))
        print('\nPrepeared train corpus for Doc2vec, {} texts'.format(len(train_corpus)))
        #print(train_corpus[500:800])
        #[6.1, 300, 200, 2] [7.4, 50, 100, 2] [6.1, 250, 100, 2] [7.7, 100, 100, 2]
        vector_size, epochs, window = 300, 200, 2
        print('Creating model with hyperparameters: vector_sizes {} epochs {} window {} ...\n'.format(vector_size, epochs, window))
        self.vectorizer = gensim.models.doc2vec.Doc2Vec(train_corpus, vector_size=vector_size, window=window, min_count=1, workers=8, epochs=epochs)
        print('Doc2vec model is ready, vocabulary {}'.format(len(self.vectorizer.wv.vocab)))
        #pprint(self.vectorizer.wv.vocab)
        
        self.train_df = df
        self.update_label_stat()

dp = DataPipeline(data_folder='/home/od13/addons/tender_cat/data/model/dump/1', 
                  trained_folder='/home/od13/addons/tender_cat/data/model/trained/1')

if 1==1:
    dp.fit()

if 1==1:
    dp.save()



dp = DataPipeline(data_folder='/home/od13/addons/tender_cat/data/model/dump/1', 
                  trained_folder='/home/od13/addons/tender_cat/data/model/trained/1')
dp.load()
tdf = dp.train_df


sdf = tdf.groupby('label_id').nunique()
pprint(sdf)

def text_label(dp_self, txt, stack=20, sim_pow=3, debug=False):
        
        sims = dp_self.most_similar(txt, topn=stack)
        
        labels_cnt = {}
        labels_sim = {}
        
        for sim in sims:
            try:
                label_id = sim['label_id'].iloc[0]
            except:
                continue
                    
            if label_id in labels_cnt: 
                labels_cnt[label_id] += 1 
                labels_sim[label_id] += math.pow(sim['sim'], sim_pow)
                #labels_sim[label_id] += sim['sim']*sim_pow
            else:
                labels_cnt[label_id] = 1 
                labels_sim[label_id] = math.pow(sim['sim'], sim_pow)
        
        labels_score = {}
        
        for label_id in labels_cnt:
            # Fraction of particular label set
            norm_freq = labels_cnt[label_id]/dp_self.label_stat['norm_freq'][label_id]
            
            print('{} / {} = {}'.format(labels_cnt[label_id], dp_self.label_stat['norm_freq'][label_id], norm_freq))
            
            total_norm_freq = norm_freq #/self.label_stat['norm_freq'][label_id]
            
            # Look at similarity
            #score = total_norm_freq*labels_sim[label_id]
            #score = labels_cnt[label_id]*dp_self.label_stat['norm_freq'][label_id]* dp_self.label_stat['inverse_freq'][label_id]
            score = (0.05*labels_cnt[label_id])*(labels_sim[label_id]/labels_cnt[label_id])*dp_self.label_stat['norm_freq'][label_id]*dp_self.label_stat['inverse_freq'][label_id]
            
            labels_score[label_id] = score
            
        sorted_labels = sorted(labels_score, key=labels_score.get, reverse=True)
        if sorted_labels:
            winner_id = sorted_labels[0]
            vals = {
                'label_id': winner_id,
                'label_name': dp_self.label_stat['name'][winner_id],

            }
        else:
            winner_id = 1001
            vals = {
                'label_id': winner_id,
                'label_name': dp_self.label_stat['name'][winner_id],

            }
            
        
        if debug:
            sims_name = {}
            for w in sorted(labels_score, key=labels_score.get, reverse=True):
                sims_name[dp_self.label_stat['name'][w]] = labels_score[w]
            vals.update({
                'sims': labels_score, 
                'sims_named': sims_name, 
                'sim':  labels_score[winner_id] if labels_score else 1,
            })
        
        return vals

label_freq, label_names, label_norm_freq, label_inverse_freq = {}, {}, {}, {}
max_count, total_cnt, major_cnt = 0, 0, 0
for i, row in tdf.groupby('label_id').nunique().iterrows():
    cnt = row[dp.train_column]
    label_freq[i] = cnt
    total_cnt += cnt
    if cnt > max_count:
        max_count = cnt
    if i==1001:
        major_cnt += 1
    label_names[i] = tdf.loc[tdf['label_id'] == i, 'label_name'].iloc[0]
    # calc unique texts count instead
    #print(max_count)

major = total_cnt/(total_cnt - major_cnt)
    
for label_id in label_freq:
    if label_id==1001:
        label_norm_freq[label_id] = label_freq[label_id]/(max_count*major)
    else:
        label_norm_freq[label_id] = label_freq[label_id]/max_count
    
    label_inverse_freq[label_id] = math.log(1) + total_cnt/label_freq[label_id]

dp.label_stat = {
    'freq': label_freq,
    'inverse_freq': label_inverse_freq,
    'norm_freq': label_norm_freq,
    'name': label_names,
    'total_cnt': total_cnt,
}

pprint(dp.label_stat['inverse_freq'])


Loaded 88 stop words
Loaded 200 stop texts from /home/od13/addons/tender_cat/data/model/trained/1/0stop_text.txt
Total 198 stop texts is now in piplene
Loaded 1956 training texts from 4 files
Character counts in texts: min 3, max 1878,  mean 146.45910020449898

Deleted 1 long texts
Deleted 1 short texts
Deleted 75 stop-texts from training corpus
Found 2271 unique tokens in texts
476 rare words added to stop list
Texts normalized again, stop-words removed from texts
Now 1796 tokens in vocabulary

24 2-grmas found
Now 1814 tokens in vocabulary

7 3-grmas found
Now 1838 tokens in vocabulary

Prepeared train corpus for Doc2vec, 1819 texts
Creating model with hyperparameters: vector_sizes 300 epochs 200 window 2 ...

Doc2vec model is ready, vocabulary 1834
Model saved to /home/od13/addons/tender_cat/data/model/trained/1/deep_sim.mod
Loaded 88 stop words
Loaded 200 stop texts from /home/od13/addons/tender_cat/data/model/trained/1/0stop_text.txt
Total 198 stop texts is now in piplene
Model lo

In [42]:

#test_sent = 'Копія свідоцтва про реєстрацію платника ПДВ або витягу з реєстру платників ПДВ або копія свідоцтва сплати єдиного податку або копія витягу з реєстру платників єдиного податку'
#test_sent = 'У разі якщо учасник, відповідно до норм чинного законодавства не є платником податку на додану вартість або єдиного податку, такий учасник подає довідку в довільній формі із зазначенням системи оподаткування, яку він обрав, подає підтверджуючі документи (у разі наявності) та зазначає інформацію про законодавчі підстави для їх ведення.'
#test_sent = 'Довідка у довільній формі про досвід виконання аналогічного (них) договору (ів) за 2020, що відповідають'
#test_sent = 'В разі отримання мотивованої відмови Замовника від підписання Акту виконаних робіт (наданих послуг) Сторонами складається протокол, в якому вказуються зауваження і терміни їх усунення, обов\'язкові для Виконавця.'
#test_sent = 'Довідка у вигляді електронного документу із ЕЦП КЕП особи, яка уповноважена на підписання такої довідки або сканкопія папурової  довідки або сканкопія нотаріально завіреної довідки про те, що службова (посадова) особа переможця процедури закупівлі, яка підписала тендерну пропозицію, не знятої чи не погашеної судимості  не має.'
#test_sent = 'Погоджений учасником проект договору згідно Додатку 3 Оголошення.'
#test_sent = 'Інформація в довільній формі за власноручним підписом фізичної особи, яка є учасником та завірена печаткою (у разі наявності) про те, що фізична особа, яка є учасником, не була засуджена за злочин, учинений з корисливих мотивів (зокрема, повязаний з хабарництвом та відмиванням коштів), судимість з якої не знято або не погашено у встановленому законом порядку'
#test_sent = 'довідку про вжиття заходів із захисту довкілля під час надання послуг згідно предмету закупівлі'
#test_sent = 'копії Статуту або іншого установчого документу (зі змінами), за підписом уповноваженої особи учасника та печаткою (за наявності).'
#test_sent = 'Вимоги до статуту - статут повинен містити відмітку державного реєстратора про проведення державної реєстрації (у випадку відсутност.відмітки державного реєстратора Учасник повинен надати інформацію з кодом доступу до результатів надання адміністративних послуг у сфері державної реєстрації, за яким існує можливіст.переглянути електронну версію документу (ів))'
#test_sent = '- довідки про присвоєння-ідентифікаційного коду (для фізичних осіб), за підписом уповноваженої особи учасника.'
#test_sent = '- листа-згоди уповноваженої (посадової) особи (осіб) учасника, що підписала документи тендерної пропозиції, який підтверджує дозвіл на використання, розповсюдження і доступ до персональних даних згідно з нормами законодавства України (у довільній формі)'
#test_sent = 'Повноваження щодо підпису документів тендерної пропозиції уповноваженої особи учасника процедури закупівлі підтверджується: для посадових (службових) осіб учасника, які уповноважені підписувати документи пропозиції та вчиняти інші юридично значущі дії від імені учасника на підставі положень установчих документів – розпорядчий документ про призначення (обрання) на посаду відповідної особи (наказ про призначення та  або протокол зборів засновників, тощо), з наданням копії паспорту уповноваженої особи; для осіб, що уповноважені представляти інтереси учасника під час проведення процедури закупівлі, та які не входять до кола осіб, що представляють інтереси учасника без довіреност.– довіреність, оформлена у відповідност.до вимог чинного законодавства, із зазначенням повноважень повіреного, разом з документами, що у відповідност.до цього пункту підтверджують повноваження посадової (службової) особи учасника, що підписала від імені учасника вказану довіреність.'
#test_sent = 'Умови, форма та зміст банківської гарантії повинні відповідати вимогам Положення про порядок здійснення банками операцій за гарантіями в національній та іноземних валютах, затвердженим Постановою Правління Національного банку України від 15.12.2004 №639, та містити наступні реквізити умови:'
#test_sent = 'Вимога щодо засвідчення того чи іншого документу тендерної пропозиції власноручним підписом учасника уповноваженої не застосовується до документів (матеріалів та інформації), що подаються у складі тендерної пропозиції, якщо такі документи (матеріали та інформація) надані учасником у формі електронного документа через електронну систему закупівель із накладанням кваліфікованого електронного підпису на кожен з таких документів (матеріал чи інформацію).'
#test_sent = 'інформації та документів, що підтверджують відповідність учасника кваліфікаційним критеріям;'
#test_sent = 'Підпис керівника або уповноваженої особи Учасника - юридичної особи, фізичної особи – підприємця'

#test_sent = 'г) система оподаткування;'
#test_sent = 'Інформація про відсутність підстави, надається в довільній формі за підписом уповноваженої особи учасника та завірену печаткою (у разі використання)'
#test_sent = '2) наявність працівників відповідної кваліфікації, які мають необхідні знання та досвід;'
#test_sent = 'Протокол №51 від 10.10.2020 р.'
#test_sent = '- не розливу нафтопродуктів, мастил та інших хімічних речовин на грунт, асфальтове покриття;'
#test_sent = '- під час експлуатації автотранспорту викид відпрацьованих газів не повинен перевищувати допустимі норми;'
#test_sent = '2) наявність працівників відповідної кваліфікації, які мають необхідні знання та досвід;'
#test_sent = 'Ініціювати внесення змін у Договір підряду, вимагати розірвання Договору підряду та відшкодування збитків за наявності істотних порушень «Підрядником» умов Договору підряду.'
test_sent = 'Ціна цього Договору може бути зменшена за взаємною згодою Сторін.'
#est_sent = 'Інші умови'
#test_sent = 'Умовами цієї документації не встановлено поділ предмета закупівлі на окремі частини (лоти).'
#test_sent = 'Замовник протягом одного робочого дня з дня їх оприлюднення зобовязаний надати розяснення на звернення учасників спрощеної закупівлі, які оприлюднюються в електронній системі закупівель, та або внести зміни до оголошення про проведення спрощеної закупівлі, та або вимог до предмета закупівлі'
#test_sent = 'Пропозиція подається в електронному вигляді шляхом заповнення електронних форм з окремими полями, у яких зазначається інформація про ціну, інші критерії оцінки (у разі їх установлення замовником), та завантаження файлів з:'
#test_sent = 'Заходи щодо захисту довкілля, які повинні бути обєднані в інформативну довідку:'

#test_sent = 'Повідомлення про відміну закупівлі автоматично надсилається всім учасникам електронною системою закупівель в день його оприлюднення.'
#test_sent = '8) зміни умов у звязку із застосуванням положень частини шостої статті 41 Закону.'
#test_sent = 'Пропозиція подається в електронному вигляді шляхом заповнення електронних форм з окремими полями, у яких зазначається інформація про ціну, інші критерії оцінки (у разі їх установлення замовником), та завантаження файлів з:'
#test_sent = 'Ціна пропозиції учасника (договірна ціна) - сума за яку учасник пропонує виконати перелік робіт, передбачених в технічній частині документації.'
#test_sent = '1) пропозиція учасника не відповідає умовам, визначеним в оголошенні про проведення спрощеної закупівлі, та вимогам до предмета закупівлі;'

if 1==1:        
    #sim = dp.text_label(test_sent, stack=20, sim_pow=3, debug=True)
    sim = text_label(dp, test_sent, stack=5, sim_pow=3, debug=True)

    print('Winner: {} {}'.format(sim['label_id'], sim['label_name']))
    sim_names = sim['sims_named']
    for w in sorted(sim_names, key=sim_names.get, reverse=True):
        print('{} {}'.format(sim_names[w], w))

    print('\n{}\n'.format(dp.normalize(test_sent)))

    sims = dp.most_similar(test_sent, topn=20)

    for sim in sims:
        try:
            print('{} -- {} -----------------------\n'.format(sim['sim'], sim['label_name'].iloc[0]))
            print('{}\n'.format(sim['text'].iloc[0]))
            print('{}\n'.format(sim['train_text'].iloc[0]))
        except:
            continue

2 / 0.9991204925241866 = 2.0017605633802815
1 / 0.016908212560386472 = 59.142857142857146
1 / 0.008454106280193236 = 118.28571428571429
1 / 0.06642512077294686 = 15.054545454545453
Winner: 1001 ?
0.015212311243636466 ?
0.0056771780140762755 Досвід виконання аналогічного договору
0.005601879200456726 ПРОЕКТ ДОГОВІРУ
0.005229772588199367 Кваліфікаційні довідки

['цін', 'договір', 'зменшен', 'взаємн', 'згод', 'сторін']

0.5117548704147339 -- ? -----------------------

У випадку виникнення спорів або розбіжностей Сторони зобовязуються вирішувати їх шляхом взаємних переговорів та консультацій, пошуку взаємоприйнятних рішень, продовження строків врегулювання розбіжностей.

випадк виникненн спор аб розбіжност сторон шлях взаємн переговор пошук рішен продовженн строк врегулюванн розбіжност

0.4458391070365906 -- Досвід виконання аналогічного договору -----------------------

Учаснику у складі тендерної пропозиції необхідно надати письмову згоду з істотними умовами договору, а також заповнений 

In [29]:
import csv 
def transform(dp_self, input_file=None, output_file=None, trained_folder=None, a=None):

    # Load input csv file
    df = pd.read_csv(input_file)
    df.fillna(value={'label_id': 1001, 'label_name': '?'}, inplace=True)
    print('Loaded {} sentences, {} files'.format(df.shape[0], len(set(df['file'].dropna()))))
    
    ################################################
    # Characters count
    df['chr_count'] = df.apply(lambda row: len(row['text']), axis=1)

    stat = {'chr_count_max': df['chr_count'].max(), 'chr_count_min': df['chr_count'].min() }

    print('Character counts in texts: min {}, max {},  mean {}\n'.format(stat['chr_count_min'], stat['chr_count_max'], df['chr_count'].mean()))

    ################################################
    # Delete too long sentences, max chr count -1
    start_len = df.shape[0]
    df = df.drop(df[df.chr_count >= stat['chr_count_max']-1].index)
    print('Deleted {} long texts'.format(start_len-df.shape[0]))

    if 0==1:
        top_n = 5
        print('\nTop {} most long now: '.format(top_n))
        print(df.sort_values('chr_count', ascending = False)['text'].head(top_n))

    ################################################
    # Delete too short sentences, chr count == min
    start_len = df.shape[0]
    max_chr_count = max(3, stat['chr_count_min'])
    df = df.drop(df[df.chr_count <= max_chr_count].index)
    print('Deleted {} short texts'.format(start_len-df.shape[0]))

    if 0==1:
        top_n = df.shape[0]//50
        print('\nTop {} most short now: '.format(top_n))
        print(df.sort_values('chr_count', ascending = True)['text'].head(top_n))
            
    ##############################################    
    # Generate normalized text, iter 1
    df['norm_text'] = df.apply(lambda row: str(' '.join(dp_self.normalize(row['text'])).strip()), axis=1)
        
    # We may unload most frequent texts to disk here
        
    ##############################################    
    # Delete stop_texts
    start_len = df.shape[0]
    df = df.drop(df[df['norm_text'].isin(dp_self.stop_text)].index)
    print('Deleted {} stop-texts from training corpus'.format(start_len-df.shape[0]))           

        
    ##############################################    
    # Words in texts
    word_list = []
    for txt in dp_self.text_src(df,'norm_text'):
        for word in txt:
            word_list.append(word)
    word_df = pd.DataFrame(word_list, columns=['word'])
    print('Found {} unique tokens in texts'.format(word_df['word'].nunique()))

    if 0==1:
        top_n = 300
        print('\nTop {} most frequent tokens:'.format(top_n))
        dp_self.print_most_freq(word_df, 'word', top_n)

    if 0==1:
        top_n = 200
        print('\nTop {} rare tokens:\n'.format(top_n))
        dp_self.print_most_freq(word_df, 'word', top_n, reverse=False)
            
    ##############################################    
    # Add rare words to stop list
    freq_word_dct = dict(word_df['word'].value_counts())
    cnt = 0            
    for w in freq_word_dct:
        if freq_word_dct[w] == 1:
            dp_self.stop_words.append(w)
            cnt += 1
    print('{} rare words added to stop list'.format(cnt))

    ##############################################    
    # Remove stop words from norm texts, normalize text, iter 2
    if 1==1:
        df['norm_text'] = df.apply(lambda row: ' '.join(dp_self.normalize(row['text'])), axis=1)
        print('Texts normalized again, stop-words removed from texts')
        word_df  = dp_self.calc_word_count(df, 'norm_text')
        print('Now {} tokens in vocabulary'.format(dp_self.word_count)) 
    
    
    
    with open(output_file, mode='w') as file:
        csv_fields = ['label_id', 'label_name', 'text', 'text_id', 'file', 'tender']
        writer = csv.DictWriter(file, fieldnames=csv_fields, delimiter=',')
        writer.writeheader()
        print('Start classifier')
        label_cnt = 0
        label_set = set()
        topn = 20000

        for i, row in df.iterrows(): 
            text = row['norm_text']
            if not isinstance(text, str):
                continue
                
            sim = dp.text_label(text, stack=20, sim_pow=2, debug=False)
            
            if 0==1:
                print('\n------------------')
                print('{}'.format(text))
                print('{}\n'.format(dp.normalize(text)))

                print('Winner: {} {}'.format(sim['label_id'], sim['label_name']))
                sim_names = sim['sims_named']
                for w in sorted(sim_names, key=sim_names.get, reverse=True):
                    print('{} {}'.format(sim_names[w], w))
            
            
            label_id = sim['label_id']
            label_name = sim['label_name']
            
            if not label_name == '?':
                label_cnt += 1
                label_set.add(label_name)

            val = {'label_id': '' if label_name == '?' else label_id,
                   'label_name': '' if label_name == '?' else label_name,
                   'text': text,
                   'text_id': df['text_id'][i],
                   }
            writer.writerow(val)
            
            topn -= 1
            if topn == 0:
                break

        print('Labeled {} sentences with {} labels'.format(label_cnt, len(label_set)))



    
%time transform(dp, input_file='/home/od13/addons/tender_cat/data/model/run/1/29input.csv', output_file='/home/od13/addons/tender_cat/data/model/run/1/29output.csv')


Loaded 656 sentences, 1 files
Character counts in texts: min 2, max 1619,  mean 146.6859756097561

Deleted 1 long texts
Deleted 2 short texts
Deleted 26 stop-texts from training corpus
Found 848 unique tokens in texts
1 rare words added to stop list
Start classifier
Labeled 334 sentences with 23 labels
CPU times: user 1min 30s, sys: 3min 40s, total: 5min 11s
Wall time: 39.8 s
